In [7]:
!pip install pyarrow


In [5]:
!pip install -q transformers peft accelerate datasets boto3


  error: subprocess-exited-with-error
  
  × Building wheel for pyarrow (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [829 lines of output]
      /tmp/pip-build-env-_l64gr7s/overlay/lib/python3.10/site-packages/setuptools/config/_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
      !!
      
              ********************************************************************************
              Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
      
              By 2026-Feb-18, you need to update your project and remove deprecated calls
              or your builds will no longer be supported.
      
              See https://packaging.python.org/en/latest/guides/writing-pyproject-toml/#license for details.
              ******************************************************************

In [ ]:
import os
import json
from dataclasses import dataclass
from typing import Dict, List

import random
import numpy as np
import torch
from torch.utils.data import Dataset

import boto3  # S3 안 쓰면 그대로 둬도 되고, 나중에 주석 처리해도 됩니다.

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    default_data_collator,
)

from peft import LoraConfig, get_peft_model


In [ ]:
@dataclass
class TrainConfig:
    # 1) 모델 & 데이터
    model_name_or_path: str = "meta-llama/Meta-Llama-3-8B-Instruct"  # 원하는 베이스 LLM 이름
    train_jsonl: str = "s3://my-bucket/path/to/train.jsonl"          # 또는 "/home/ec2-user/data/train.jsonl"
    
    # 결과 저장 위치: EBS 볼륨 내 현재 작업 디렉토리 기준 "./model"
    output_dir: str = "./model"

    # 2) 토크나이즈 & 길이
    max_length: int = 1024

    # 3) 학습 하이퍼파라미터
    per_device_train_batch_size: int = 1
    gradient_accumulation_steps: int = 16
    num_train_epochs: int = 3
    learning_rate: float = 2e-4
    warmup_ratio: float = 0.03

    # 4) 로깅 & 저장
    logging_steps: int = 50
    save_steps: int = 500
    seed: int = 42

    # 5) 기타
    dataloader_num_workers: int = 2  # Colab이면 0~2 정도로 유지


cfg = TrainConfig()
cfg


In [ ]:
def resolve_jsonl_path(path: str) -> str:
    """
    - s3://bucket/key 형식이면 /tmp/train.jsonl 로 다운로드 후 해당 경로 반환
    - 그 외에는 로컬 경로 그대로 반환
    """
    if path.startswith("s3://"):
        no_scheme = path[5:]
        bucket, key = no_scheme.split("/", 1)

        local_path = "/tmp/train.jsonl"
        os.makedirs(os.path.dirname(local_path), exist_ok=True)

        s3 = boto3.client("s3")
        s3.download_file(bucket, key, local_path)
        print(f"[INFO] downloaded {path} -> {local_path}")
        return local_path
    else:
        return path


def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


seed_everything(cfg.seed)


In [ ]:
class JsonlSftDataset(Dataset):
    """
    JSONL SFT 포맷:
      {"prompt": "...", "completion": "...", "date": "..."} 한 줄씩

    full_text = prompt_text + "\\n\\n### 답변:\\n" + completion
    을 만들고, prompt_text + "### 답변:" 구간까지는 labels를 -100으로 마스킹.
    """

    def __init__(self, jsonl_path: str, tokenizer, max_length: int = 1024):
        self.tokenizer = tokenizer
        self.max_length = max_length

        self.records: List[Dict] = []
        with open(jsonl_path, "r", encoding="utf-8") as f:
            for line in f:
                obj = json.loads(line)
                prompt = obj["prompt"]
                completion = obj["completion"]
                self.records.append(
                    {
                        "prompt": prompt,
                        "completion": completion,
                        "date": obj.get("date", None),
                    }
                )

    def __len__(self):
        return len(self.records)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        rec = self.records[idx]
        prompt = rec["prompt"]
        completion = rec["completion"]

        # 1) 프롬프트 텍스트 / 전체 텍스트 구성
        #    여기까지가 "입력" 역할
        prompt_text = prompt.rstrip() + "\n\n### 답변:\n"
        full_text = prompt_text + completion.rstrip()

        # 2) prompt 길이(토큰 개수) 계산 - special tokens 제외
        prompt_ids = self.tokenizer(
            prompt_text,
            add_special_tokens=False,
        )["input_ids"]
        prompt_len = len(prompt_ids)

        # 3) full_text 토크나이즈 (special tokens 포함)
        encoded = self.tokenizer(
            full_text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_length,
        )

        input_ids = encoded["input_ids"]
        attn_mask = encoded["attention_mask"]

        # 4) labels = input_ids 복사 후, prompt 부분 마스킹
        labels = input_ids.copy()

        # BOS 토큰이 앞에 붙어 있는 경우, 그만큼 추가로 마스킹
        bos_extra = 0
        if hasattr(self.tokenizer, "bos_token_id") and self.tokenizer.bos_token_id is not None:
            if len(input_ids) > 0 and input_ids[0] == self.tokenizer.bos_token_id:
                bos_extra = 1

        mask_len = min(len(labels), prompt_len + bos_extra)
        for i in range(mask_len):
            labels[i] = -100

        return {
            "input_ids": torch.tensor(input_ids, dtype=torch.long),
            "attention_mask": torch.tensor(attn_mask, dtype=torch.long),
            "labels": torch.tensor(labels, dtype=torch.long),
        }


In [ ]:
# 1) 토크나이저
tokenizer = AutoTokenizer.from_pretrained(
    cfg.model_name_or_path,
    use_fast=True,
)

# pad 토큰 세팅 (없는 모델들 대비)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 2) 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    cfg.model_name_or_path,
    torch_dtype=torch.float16,
    device_map="auto",  # GPU 있으면 GPU, 없으면 CPU
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False  # 학습 시에는 False 권장

# 3) LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    # LLaMA 계열 기준. 다른 모델이면 target_modules 이름만 바꿔주면 됨.
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


In [ ]:
# S3 또는 로컬 경로를 실제 로컬 파일로 resolve
train_jsonl_local = resolve_jsonl_path(cfg.train_jsonl)
print("Train JSONL local path:", train_jsonl_local)

# Dataset 생성
train_dataset = JsonlSftDataset(
    jsonl_path=train_jsonl_local,
    tokenizer=tokenizer,
    max_length=cfg.max_length,
)

# collator: 우리가 만든 labels를 그대로 유지해야 하므로 default_data_collator 사용
data_collator = default_data_collator


In [ ]:
# trainer
from transformers import TrainingArguments

os.makedirs(cfg.output_dir, exist_ok=True)  # EBS 상의 ./model 폴더 생성

training_args = TrainingArguments(
    output_dir=cfg.output_dir,
    overwrite_output_dir=True,

    num_train_epochs=cfg.num_train_epochs,
    per_device_train_batch_size=cfg.per_device_train_batch_size,
    gradient_accumulation_steps=cfg.gradient_accumulation_steps,
    learning_rate=cfg.learning_rate,
    warmup_ratio=cfg.warmup_ratio,

    logging_steps=cfg.logging_steps,
    save_steps=cfg.save_steps,
    save_total_limit=3,

    fp16=True,
    bf16=False,

    seed=cfg.seed,
    report_to="none",  # W&B 등을 쓸 거면 "wandb"로 변경
    dataloader_num_workers=cfg.dataloader_num_workers,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

print("Trainer ready.")


In [ ]:
trainer.train()

In [ ]:
trainer.save_model(cfg.output_dir)      # LoRA 어댑터 포함한 모델 저장
tokenizer.save_pretrained(cfg.output_dir)

print("LoRA fine-tuning 완료. 저장 위치:", cfg.output_dir)


In [4]:
#simple test
from peft import PeftModel

# 베이스 모델 다시 로드 (새 세션이거나, 검증용)
base_model = AutoModelForCausalLM.from_pretrained(
    cfg.model_name_or_path,
    torch_dtype=torch.float16,
    device_map="auto",
)
base_model.config.pad_token_id = tokenizer.pad_token_id
base_model.config.use_cache = True  # inference에서는 True로 켜도 됩니다.

lora_model = PeftModel.from_pretrained(
    base_model,
    cfg.output_dir,
)
lora_model.eval()

def generate_comment(prompt_text: str, max_new_tokens: int = 256):
    inputs = tokenizer(
        prompt_text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=cfg.max_length,
    ).to(lora_model.device)

    with torch.no_grad():
        outputs = lora_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id,
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


test_prompt = """당신은 한국 주식 애널리스트다.

[종목] 005930
[기간] 2025-12-05

[OHLCV 요약]
- 시가: ...
- 고가: ...
- 저가: ...
- 종가: ...
- 거래량: ...

[VQ 코드 시퀀스]
[12, 45, 78, 90]

[뉴스 헤드라인]
- "삼성전자, XXX 관련 수주 기대"

위 정보를 종합하여 2~3문장 분량의 시황 코멘트를 작성하라.

### 답변:
"""

print(generate_comment(test_prompt))
